# Получение данных о домах города

In [1]:
import requests as rq
import pandas as pd
import json
from dadata import Dadata
from bs4 import BeautifulSoup as bs

## ШАГ 0. Создаём функции

In [79]:
# Обработка результатов от dadata.ru
def create_line(result):
    line1 = {}
    line1.update({'value':result[0]['value']})
    line1.update({'unrestricted_value':result[0]['unrestricted_value']})
    for keys in result[0]['data'].keys():
        line1.update({keys:result[0]['data'][keys]})
    return line1

In [134]:
# Получение кодов 
def parser(site):
    source = rq.get(site).content
    soup = bs(source, 'html.parser')

    table_label = soup.findAll('th')
    table_data = soup.findAll('td')

    table_data_list = []
    for item in list(table_data):
        table_data_list.append(item.string)
    result = {}
    for k in range(0, len(table_data_list), 6):
        result.update({table_data_list[k]:table_data_list[k+2:k+6]})

    return result

In [65]:
def houses_by_street(site):
    source = rq.get(site).text
    soup = bs(source)
    href = []
    for item in soup.findAll('a', class_="kladrs-objects__number"):
        attr = item.attrs
        href.append(attr['href'])
    return href

## ШАГ 1. Получаем коды улиц города

alta.ru Улицы по коду города

In [46]:
source = rq.get('https://www.alta.ru/fias/73b29372-242c-42c5-89cd-8814bc2368af/').text
soup = bs(source)
href = []
for item in soup.findAll('a', class_="jFastSearch_key"):
    attr = item.attrs
    href.append(attr['href'])
fias_streets = []
for item in href:
    fias_streets.append(item[6:-1])
del([item, soup, source, href])

## ШАГ 2. Получаем коды домов по кодам улиц

alta.ru Дома по коду улицы

In [66]:
fias_houses = []
for street_code in fias_streets:
    site = 'https://www.alta.ru/fias/' + street_code + '/'
    fias_houses.append(houses_by_street(site))

In [69]:
fias_houses_all = []
for item in fias_houses:
    for elem in item:
        fias_houses_all.append(elem)

In [71]:
fias_houses_codes = []
for item in fias_houses_all:
    fias_houses_codes.append(item[6:-1])

## ШАГ 3. Получаем данные по кодам домов

dadata.ru Поиск данных по коду дома

In [74]:
token = str(open('E:/CODE/BUILDINGS_OF_DIMITROVGRAD/access_3.txt', 'r').read())
dadata = Dadata(token)

full_info = []
for code in fias_houses_codes:
    result = dadata.find_by_id("address", code)
    full_info.append(create_line(result))

## Дом МинЖКХ

In [139]:
result_list = []
for i in range(11):
    number_page = i+1
    site = 'https://dom.mingkh.ru/ulyanovskaya-oblast/dimitrovgrad/houses?page=' + str(number_page)
    result_list.append(parser(site))

result_0 = {}
for item in result_list:
    result_0.update(item)

pd.DataFrame(result_0).T.to_csv('ready.csv', encoding='cp1251')